<a href="https://colab.research.google.com/github/curtiscu/LYIT/blob/master/BeatTrackingTests_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Work out beats, timings, etc for loaded data

Doc that came in handy creating this notebook..

* https://stackoverflow.com/questions/48905127/importing-py-files-in-google-colab
* https://ipython.org/ipython-doc/stable/config/extensions/autoreload.html
* https://switowski.com/blog/ipython-autoreload


# Setup env


In [0]:
def set_pandas_display_options() -> None:
    # Ref: https://stackoverflow.com/a/52432757/
    display = pd.options.display

    display.max_columns = 1000
    display.max_rows = 100
    display.max_colwidth = 199
    display.width = None
    # display.precision = 2  # set as needed

set_pandas_display_options()

In [63]:
# print all cell output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# install required libs
!pip install mido


LOADING - data_prep.py module name is: data_prep


In [64]:
# mount google drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [65]:
# test, peek at data
! ls -al '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/'

# test, modules from local  'E:\Google Drive\LYIT\Dissertation\modules'
! ls -al '/content/drive/My Drive/LYIT/Dissertation/modules/'

total 35
-rw------- 1 root root 2589 Apr 27 12:01 10_soul-groove10_102_beat_4-4.mid
-rw------- 1 root root 4793 Apr 27 12:01 1_funk-groove1_138_beat_4-4.mid
-rw------- 1 root root 3243 Apr 27 12:01 2_funk-groove2_105_beat_4-4.mid
-rw------- 1 root root 4466 Apr 27 12:01 3_soul-groove3_86_beat_4-4.mid
-rw------- 1 root root 2551 Apr 27 12:01 4_soul-groove4_80_beat_4-4.mid
-rw------- 1 root root 3798 Apr 27 12:01 5_funk-groove5_84_beat_4-4.mid
-rw------- 1 root root 3760 Apr 27 12:01 6_hiphop-groove6_87_beat_4-4.mid
-rw------- 1 root root 1894 Apr 27 12:01 7_pop-groove7_138_beat_4-4.mid
-rw------- 1 root root 2437 Apr 27 12:01 8_rock-groove8_65_beat_4-4.mid
-rw------- 1 root root 3448 Apr 27 12:01 9_soul-groove9_105_beat_4-4.mid
total 10
-rw------- 1 root root 5785 May 10 22:10 data_prep.py
drwx------ 2 root root 4096 May 10 22:11 __pycache__


## Auto reload module

I'm now using a module edited and saved to google drive which is automatically pushed to the cloud and made available to the colab env. Changes need time to propagate, and imports don't 'reimport' to load changes, so trying the following...

Note the code below is not very reliable, it seems to work occasionally, after some time, but haven't been able to work out the pattern to it. 

If in a hurry, brute force loading of changes by restarting the runtime.

In [66]:
# tool to auto reload modules.
%load_ext autoreload

# config to auto-reload all modules, handy to make 
# writing and testing modules much easier.
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
# imports
import pandas as pd
import mido 
from mido import MidiFile

import datetime

# import my modules
import sys
sys.path.append('/content/drive/My Drive/LYIT/Dissertation/modules/')
import data_prep



In [68]:
print(datetime.datetime.now())

2020-05-10 22:11:25.317404


In [69]:
# testing auto reload of modules 
data_prep.test_function_call('bling')

test function called worked! :)  bling


# Test creating object from custom module

In [70]:
midi_file = data_prep.MIDI_File_Wrapper('/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/10_soul-groove10_102_beat_4-4.mid')

__ FILE: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/10_soul-groove10_102_beat_4-4.mid
tracks: [<midi track 'MIDI' 639 messages>]
time sig: <meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
tempo: <meta message set_tempo tempo=588235 time=0>


... the above verifies I'm able to create custom objects from custom code, great!


# Brainstorming ways to calculate beat times

Things I might need to calculate..
* start/ end times of bars
* beat times for 1/4, 1/8, 1/16, 1/32 notes
* shuffles in 4/4?

Ways to go about it...
* Take list of MIDI note_on, find what beat & bar to associate each beat with.
* For a given musical beat, take the acceptable timing range for something to be considered related to it, filter all the MIDI notes against this range to see which if any fall into that range.

Python code/ libs for ranges and splitting datasets..
* python 'range', only works on integers
* pandas.cut, pandas.qcut
* numpy
* itertools 

Some handy refs...
* https://realpython.com/how-to-use-numpy-arange/
* https://numpy.org/doc/stable/reference/generated/numpy.linspace.html
* https://pbpython.com/pandas-qcut-cut.html < looks very promising!!!


## peek at example object data..

In [71]:
# take a peek inside what we've setup already...
midi_file.__dict__

{'df_midi_data':             msg_type  delta_time  ... ticks_since_start  seconds_since_start
 0         track_name           0  ...                 0             0.000000
 1    instrument_name           0  ...                 0             0.000000
 2     time_signature           0  ...                 0             0.000000
 3      key_signature           0  ...                 0             0.000000
 4       smpte_offset           0  ...                 0             0.000000
 ..               ...         ...  ...               ...                  ...
 634          note_on          37  ...             30566            37.458315
 635         note_off          83  ...             30649            37.560030
 636          note_on          24  ...             30673            37.589442
 637         note_off          83  ...             30756            37.691158
 638     end_of_track           0  ...             30756            37.691158
 
 [639 rows x 5 columns],
 'my_file_midi': <mid

In [72]:
midi_file.__doc__

' utility wrapper for loading, parsding a mido.MidiFile object'

## Calulate some time metrics


In [91]:
print('file name: {}'.format(midi_file.my_file_name))
print('ticks per quarter note: {}'.format(midi_file.ticks()))
print('tempo (microseconds p/ quarter): {}'.format(midi_file.tempo_us()))
print('tempo (bpm): {}'.format(midi_file.tempo_bpm()))

file name: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/10_soul-groove10_102_beat_4-4.mid
ticks per quarter note: 480
tempo (microseconds p/ quarter): 588235
tempo (bpm): 102.0000510000255


In [95]:

def calc_sec(ticks):
  return mido.tick2second(ticks, midi_file.ticks(), midi_file.tempo_us() )

# caluate absolute time in milliseconds
# mido.tick2second(ticks, < ticks per quarter >, < tempo in microseconds> )
print('seconds per tick (ticks2seconds): {}'.format(mido.tick2second(1, midi_file.ticks(), midi_file.tempo_us() )))

# NOTE: calculate microseconds per tick
# µs_per_tick  = ticks per qaurter beat / file time in microseconds
us_per_tick =  midi_file.tempo_us()/ midi_file.ticks()
print('microseconds per tick: {}'.format(us_per_tick))

# NOTE: the following works for different time sigs, i.e. 
#   will handle 6/8  and  4/4 ok...
# beat ticks = <ticks per quarter beat> x  ( 4/ < file time sig denominator > )
ticks_per_beat = midi_file.ticks() * ( 4/ midi_file.ts_denom() )
print('per beat: {}, ticks: {}, sec: {}'.format(midi_file.ts_denom(), ticks_per_beat, calc_sec(ticks_per_beat)))

# NOTE: total ticks for a measure..
# bar ticks = < beat ticks >  x < beats per bar >
ticks_per_bar = ticks_per_beat * midi_file.ts_num()
print('per bar - ticks: {}, sec: {}'.format(ticks_per_bar, calc_sec(ticks_per_bar)))

# Ticks per 1/8 note
ticks_per_8 = midi_file.ticks() * ( 4/ 8 )
print('per 1/8 - ticks: {}, sec: {}'.format(ticks_per_8, calc_sec(ticks_per_8)))

# Ticks per 1/16 note
ticks_per_16 = midi_file.ticks() * ( 4/ 16 )
print('per 1/16 - ticks: {}, sec: {}'.format(ticks_per_16, calc_sec(ticks_per_16)))


seconds per tick (ticks2seconds): 0.0012254895833333332
microseconds per tick: 1225.4895833333333
per beat: 4, ticks: 480.0, sec: 0.588235
per bar - ticks: 1920.0, sec: 2.35294
per 1/8 - ticks: 240.0, sec: 0.2941175
per 1/16 - ticks: 120.0, sec: 0.14705875


## Calculate number of bars in file 


In [97]:

# last note_on event in file, in seconds..
#    event# 636, time(s): 37.589442
# .. divided by length of bar in seconds
total_bars = 37.589442 /2.35294
print('number bars in file: {}'.format( total_bars))

number bars in file: 15.975520837760419


In [103]:
import math

# round up number of bars
total_bars = math.ceil(total_bars)

print('# bars, rounded up: {}'.format(total_bars))

# bars, rounded up: 16


# Calculate number of beats in file

In [104]:
# number of beats in file
total_beats = total_bars * midi_file.ts_num()
print('total beatz: {}'.format(total_beats))


total beatz: 64


In [0]:
# 1/16s in file

# Ticks per 1/16 note
ticks_per_16 = midi_file.ticks() * ( 4/ 16 )

# Splitting data




## Testing pandas.cut

In [122]:
# example usage -> pd.cut(df['ext price'], bins=4)
display(pd.cut(midi_file.df_midi_data['ticks_since_start'], bins=64).to_frame())

,ticks_since_start
0,"(-30.756, 480.562]"
1,"(-30.756, 480.562]"
2,"(-30.756, 480.562]"
3,"(-30.756, 480.562]"
4,"(-30.756, 480.562]"
...,...
634,"(30275.438, 30756.0]"
635,"(30275.438, 30756.0]"
636,"(30275.438, 30756.0]"
637,"(30275.438, 30756.0]"


There's a problem using pd.cut above, note the last bucket ends at precisely the last numbered tick, but the first bucket has a start that's a negative! Looks like it works by segmenting from the high number/ end of the data down, therefore ends up -ve.


In [106]:
# count rows in each bin, example -> pd.cut(df['ext price'], bins=4).value_counts()
pd.cut(midi_file.df_midi_data['ticks_since_start'], bins=64).value_counts()

(-30.756, 480.562]        17
(22105.875, 22586.438]    16
(29794.875, 30275.438]    16
(30275.438, 30756.0]      15
(20664.188, 21144.75]     14
                          ..
(6247.312, 6727.875]       7
(2402.812, 2883.375]       7
(19703.062, 20183.625]     7
(480.562, 961.125]         7
(13936.312, 14416.875]     7
Name: ticks_since_start, Length: 64, dtype: int64